## Youtube 댓글 크롤링

In [35]:
#필요한 라이브러리 설치
#pip install selenium : 브라우저 조정을 통한 웹 테스트
#pip install bs4 : HTML,XML 데이터 Parsing
#pip install pandas : 데이터 조작,가공, 최적화, DataFrame
#pip install openpyxl : Excel 파일 Read/Write
#pip install lxml : XML,HTML 처리를 위한 기능

In [36]:
#라이브러리 전처리
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver as wd


#Check start time 시작 시간(전체 수행시간을 구하기 위해)
start=time.time()

#chrom driver경로 설정
driver=wd.Chrome(executable_path=r"C:\Users\rkdtn\Desktop\chromedriver.exe")
#driver=wd.Chrome(executable_path="C:/Users/rkdtn/Desktop/chromedriver.exe")

#댓글 수집할 Youtube URL 입력
#url="https://youtu.be/0F2GrIuSgZ8" #volvo
url="https://www.youtube.com/watch?v=NUmWkbiYnhA" #samsung
driver.get(url)

last_page_height=driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1.0)       # 인터발 1이상으로 줘야 데이터 취득가능(롤링시 데이터 로딩 시간 때문)
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    if new_page_height == last_page_height:
        break
    last_page_height = new_page_height

html_source = driver.page_source
driver.close()

# HTML 태크 크롤링 작업
soup = BeautifulSoup(html_source, "lxml")

youtube_user_IDs = soup.select("div#header-author > a > span")
youtube_comments = soup.select("yt-formatted-string#content-text")
comment_like_num=soup.select("span#vote-count-middle.style-scope")
#comment_unlike_num=soup.select("")




In [37]:
##############불필요한 공백, 탭 등의 공간을 정리####################



str_youtube_userIDs = []   # USER ID 배열
str_youtube_comments = []  # USER 댓글 내용 배열
str_comment_like_num=[]  #댓글 좋아요 갯수 배열
#str_comment_unlike_num=[]  #댓글 싫어요 갯수 배열

# REPLACE DATA
for i in range(len(youtube_user_IDs)):
    str_tmp = str(youtube_user_IDs[i].text)
    str_tmp = str_tmp.replace('\n', '')
    str_tmp = str_tmp.replace('\t', '')
    str_tmp = str_tmp.replace('   ','')
    str_youtube_userIDs.append(str_tmp)

    str_tmp = str(youtube_comments[i].text)
    str_tmp = str_tmp.replace('\n', '')
    str_tmp = str_tmp.replace('\t', '')
    str_tmp = str_tmp.replace('   ','')
    str_youtube_comments.append(str_tmp)
    
    str_tmp=str(comment_like_num[i].text)
    str_comment_like_num.append(str_tmp)
    

In [38]:
############## 보기 좋은 형태로 데이터를 변경###############

## MODIFY VIEW FORMAT
pd_data = {"ID":str_youtube_userIDs, "comment":str_youtube_comments,"like_num":str_comment_like_num}
youtube_pd = pd.DataFrame(pd_data)

In [39]:
###############엑셀파일로 저장####################
youtube_pd.to_excel("C:/Users/rkdtn/Desktop/data4.xlsx", sheet_name="sheet1", index=True)
print("Running Time : ", time.time() - start)       # 전체 수행 시킨 출력

Running Time :  85.33774638175964
